In [1]:
import arcpy
import os

# Set up your workspace
arcpy.env.workspace = r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\May 2021 ET\May 2021 ET.gdb"
output_folder = r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\May 2021 ET"

# Define the path to the Minnesota state shapefile
minnesota_state_shapefile = "mn_county_boundaries"

# Create a feature layer for the Minnesota state shapefile
arcpy.MakeFeatureLayer_management(minnesota_state_shapefile, "Minnesota_State_Layer")

# Get a list of all county names in the Minnesota state shapefile
county_names = [row[0] for row in arcpy.da.SearchCursor(minnesota_state_shapefile, "CTY_Name")]

# Iterate through each county
for county_name in county_names:
    # Define the SQL query to select the current county
    sql_query = f"CTY_Name = '{county_name}'"

    # Make a feature layer for the current county
    arcpy.MakeFeatureLayer_management("Minnesota_State_Layer", "Current_County_Layer", sql_query)

    # Define the output path for the clipped shapefile
    output_shapefile = os.path.join(output_folder, f"{county_name}_clipped.shp")

    # Perform the clip operation
    arcpy.analysis.Clip("Current_County_Layer", "Minnesota_State_Layer", output_shapefile)

    print(f"{county_name} has been clipped and saved to {output_shapefile}")

Lake of the Woods has been clipped and saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\May 2021 ET\Lake of the Woods_clipped.shp
Kittson has been clipped and saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\May 2021 ET\Kittson_clipped.shp
Roseau has been clipped and saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\May 2021 ET\Roseau_clipped.shp
Koochiching has been clipped and saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\May 2021 ET\Koochiching_clipped.shp
Marshall has been clipped and saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\May 2021 ET\Marshall_clipped.shp


ExecuteError: ERROR 000601: Cannot delete C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\May 2021 ET\St. Louis_clipped.shp.  May be locked by another application.
Failed to execute (Clip).


### Extract by mask

In [2]:
import arcpy

# Set up environment
arcpy.env.workspace = r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\May 2021 ET\May 2021 ET.gdb"
project = arcpy.mp.ArcGISProject("CURRENT")
map = project.listMaps()[0]  # Assuming you want to work with the first map

# Define the inputs
years = ["2021", "2019", "2018"]
input_rasters = ["OpenET_Ensemble_ET_May_2021_Minnesota.tif", "OpenET_Ensemble_ET_May_2019_Minnesota.tif", "OpenET_Ensemble_ET_May_2018_Minnesota.tif"]
mask_files = ["2021 corn lc.TIF", "corn lc 2019.TIF", "corn lc 2018.TIF"]
output_names = ["May2021cornET", "May2019cornET", "May2018cornET"]

# Loop through each year to process and add to the map
for input_raster, mask_file, output_name in zip(input_rasters, mask_files, output_names):
    # Process each raster
    out_raster = arcpy.sa.ExtractByMask(
        in_raster=input_raster,
        in_mask_data=mask_file,
        extraction_area="INSIDE"
    )
    # Save the raster
    out_path = arcpy.env.workspace + "\\" + output_name
    out_raster.save(out_path)
    
    # Add the saved raster to the map with a specified name
    layer = map.addDataFromPath(out_path)
    layer.name = output_name  # Set the layer name

# Save changes to the project
project.save()


In [4]:
import arcpy
import os

# Set up your workspace
arcpy.env.workspace = r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\May 2021 ET\May 2021 ET.gdb"
output_folder = r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\May 2021 ET"  # Output folder is the workspace for simplicity

# Define the path to the Minnesota county boundaries shapefile
minnesota_county_shapefile = "mn_county_boundaries"

# Years to process ET maps for
years = ["2021", "2019", "2018"]
et_map_layers = {
    "2021": "May2021cornET",
    "2019": "May2019cornET",
    "2018": "May2018cornET"
}

# Get a list of county names from the attribute table of the county shapefile
county_names = [row[0] for row in arcpy.da.SearchCursor(minnesota_county_shapefile, "CTY_Name")]

# Iterate through each year and each county
for year in years:
    et_map_layer = et_map_layers[year]  # Get the current year's ET map layer name
    for county_name in county_names:
        # Define the SQL query to select the current county
        sql_query = f"CTY_Name = '{county_name}'"

        # Make a feature layer for the current county boundary
        arcpy.MakeFeatureLayer_management(minnesota_county_shapefile, "Current_County_Layer", sql_query)

        # Overlay the county boundary with the ET map and extract the values
        county_et_map = os.path.join(output_folder, f"{county_name.replace(' ', '_')}_ET_{year}.tif")
        arcpy.sa.ExtractByMask(et_map_layer, "Current_County_Layer").save(county_et_map)

        print(f"{county_name} {year} ET map has been saved to {county_et_map}")


Lake of the Woods 2021 ET map has been saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\May 2021 ET\Lake_of_the_Woods_ET_2021.tif
Kittson 2021 ET map has been saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\May 2021 ET\Kittson_ET_2021.tif
Roseau 2021 ET map has been saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\May 2021 ET\Roseau_ET_2021.tif
Koochiching 2021 ET map has been saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\May 2021 ET\Koochiching_ET_2021.tif
Marshall 2021 ET map has been saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\May 2021 ET\Marshall_ET_2021.tif
St. Louis 2021 ET map has been saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\May 2021 ET\St._Louis_ET_2021.tif
Beltrami 2021 ET map has been saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\May 2021 ET\Beltrami_ET_2021.tif
Polk 2021 ET map has been saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\May 2021 ET\Polk_ET_2021.tif
Pennington 2

Aitkin 2019 ET map has been saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\May 2021 ET\Aitkin_ET_2019.tif
Wadena 2019 ET map has been saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\May 2021 ET\Wadena_ET_2019.tif
Crow Wing 2019 ET map has been saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\May 2021 ET\Crow_Wing_ET_2019.tif
Carlton 2019 ET map has been saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\May 2021 ET\Carlton_ET_2019.tif
Otter Tail 2019 ET map has been saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\May 2021 ET\Otter_Tail_ET_2019.tif
Wilkin 2019 ET map has been saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\May 2021 ET\Wilkin_ET_2019.tif
Pine 2019 ET map has been saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\May 2021 ET\Pine_ET_2019.tif
Todd 2019 ET map has been saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\May 2021 ET\Todd_ET_2019.tif
Morrison 2019 ET map has been saved to C:\Users\

Big Stone 2018 ET map has been saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\May 2021 ET\Big_Stone_ET_2018.tif
Sherburne 2018 ET map has been saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\May 2021 ET\Sherburne_ET_2018.tif
Swift 2018 ET map has been saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\May 2021 ET\Swift_ET_2018.tif
Kandiyohi 2018 ET map has been saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\May 2021 ET\Kandiyohi_ET_2018.tif
Wright 2018 ET map has been saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\May 2021 ET\Wright_ET_2018.tif
Anoka 2018 ET map has been saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\May 2021 ET\Anoka_ET_2018.tif
Meeker 2018 ET map has been saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\May 2021 ET\Meeker_ET_2018.tif
Lac qui Parle 2018 ET map has been saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\May 2021 ET\Lac_qui_Parle_ET_2018.tif
Washington 2018 ET map has b

In [3]:
import arcpy
import os

# Define your base workspace
base_output_folder = r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\May 2021 ET"

# Years to process and their corresponding ET map layers
years = ["2021", "2019", "2018"]
et_map_layers = {
    "2021": "May2021cornET",
    "2019": "May2019cornET",
    "2018": "May2018cornET"
}

# Path to the Minnesota county boundaries shapefile
minnesota_county_shapefile = "mn_county_boundaries"

# Get a list of county names from the attribute table of the county shapefile
county_names = [row[0] for row in arcpy.da.SearchCursor(minnesota_county_shapefile, "CTY_Name")]

# Process each year
for year in years:
    # Set up specific year's workspace
    output_folder = os.path.join(base_output_folder, f"May{year}ET")
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)  # Create the directory if it does not exist

    # Reference the layer name of the ET map for the current year in ArcGIS Pro from the dictionary
    et_map_layer = et_map_layers[year]

    # Create an empty dictionary to store county names and their average ET values for this year
    county_et_averages = {}

    # Iterate through each county
    for county_name in county_names:
        try:
            # Define the SQL query to select the current county
            sql_query = f"CTY_Name = '{county_name}'"

            # Make a feature layer for the current county boundary
            arcpy.MakeFeatureLayer_management(minnesota_county_shapefile, "Current_County_Layer", sql_query)

            # Define the output path for the clipped ET raster
            county_et_map = os.path.join(output_folder, f"{county_name.replace(' ', '_')}_ET_{year}.tif")

            # Clip the ET raster with the county boundary
            arcpy.management.Clip(et_map_layer, "#", county_et_map, "Current_County_Layer", "#", "ClippingGeometry")

            # Check if raster is valid and has data
            if int(arcpy.management.GetRasterProperties(county_et_map, "ROWCOUNT").getOutput(0)) > 0:
                # Calculate statistics for the clipped raster to ensure 'MEAN' can be computed
                arcpy.management.CalculateStatistics(county_et_map)

                # Calculate the mean of the clipped raster
                result = arcpy.GetRasterProperties_management(county_et_map, "MEAN")
                mean_value = float(result.getOutput(0))

                # Store the county name and its average ET value in the dictionary
                county_et_averages[county_name] = mean_value
            else:
                # Assign NA for empty or no data raster
                county_et_averages[county_name] = "NA"
                print(f"No valid data for {county_name} in {year}, assigned 'NA'.")

         
    print(f"{county_name} ET map for {year} has been processed and saved to {county_et_map}. Average value: {county_et_averages[county_name]}")
        except Exception as e:
            print(f"Failed to process {county_name} for {year}: {str(e)}")
            county_et_averages[county_name] = "NA"  # Assign NA in case of any errors

    # Print the dictionary containing county names and their average ET values for the year
    print(f"County-wise Average ET Values for May {year}:")
    for county, avg_et in county_et_averages.items():
        print(f"{county}: {avg_et}")

Failed to process Lake of the Woods for 2021: ERROR 001100: Failed because no statistics are available.
Failed to execute (GetRasterProperties).

Kittson ET map for 2021 has been processed and saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\May 2021 ET\May2021ET\Kittson_ET_2021.tif. Average value: 59.7416666666667
Roseau ET map for 2021 has been processed and saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\May 2021 ET\May2021ET\Roseau_ET_2021.tif. Average value: 62.1521739130435
Koochiching ET map for 2021 has been processed and saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\May 2021 ET\May2021ET\Koochiching_ET_2021.tif. Average value: 78.0
Marshall ET map for 2021 has been processed and saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\May 2021 ET\May2021ET\Marshall_ET_2021.tif. Average value: 58.3672839506173
St. Louis ET map for 2021 has been processed and saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\May 2021 ET\May2021ET\St._


Failed to process Lake for 2021: ERROR 001100: Failed because no statistics are available.
Failed to execute (GetRasterProperties).

Failed to process Lake for 2021: ERROR 001100: Failed because no statistics are available.
Failed to execute (GetRasterProperties).

Failed to process Lake for 2021: ERROR 001100: Failed because no statistics are available.
Failed to execute (GetRasterProperties).

Failed to process Lake for 2021: ERROR 001100: Failed because no statistics are available.
Failed to execute (GetRasterProperties).

Failed to process Lake for 2021: ERROR 001100: Failed because no statistics are available.
Failed to execute (GetRasterProperties).

Failed to process Cook for 2021: ERROR 001100: Failed because no statistics are available.
Failed to execute (GetRasterProperties).

Failed to process Cook for 2021: ERROR 001100: Failed because no statistics are available.
Failed to execute (GetRasterProperties).

Failed to process Cook for 2021: ERROR 001100: Failed because no sta

Lincoln ET map for 2019 has been processed and saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\May 2021 ET\May2019ET\Lincoln_ET_2019.tif. Average value: 65.203166226913
Lyon ET map for 2019 has been processed and saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\May 2021 ET\May2019ET\Lyon_ET_2019.tif. Average value: 66.4489795918369
Le Sueur ET map for 2019 has been processed and saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\May 2021 ET\May2019ET\Le_Sueur_ET_2019.tif. Average value: 69.8545454545454
Rice ET map for 2019 has been processed and saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\May 2021 ET\May2019ET\Rice_ET_2019.tif. Average value: 63.4599589322381
Brown ET map for 2019 has been processed and saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\May 2021 ET\May2019ET\Brown_ET_2019.tif. Average value: 70.0889679715303
Nicollet ET map for 2019 has been processed and saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\May


Failed to process Cook for 2019: ERROR 001100: Failed because no statistics are available.
Failed to execute (GetRasterProperties).

Failed to process Lake for 2019: ERROR 001100: Failed because no statistics are available.
Failed to execute (GetRasterProperties).

Failed to process Lake for 2019: ERROR 001100: Failed because no statistics are available.
Failed to execute (GetRasterProperties).

County-wise Average ET Values for May 2019:
Lake of the Woods: 69.5
Kittson: 45.9126213592233
Roseau: 54.7313432835821
Koochiching: 55.0625
Marshall: 52.551094890511
St. Louis: 74.0
Beltrami: 76.4827586206897
Polk: 58.7417102966841
Pennington: 59.4810126582278
Cook: NA
Lake: NA
Clearwater: 81.8409090909091
Red Lake: 64.4714285714286
Itasca: 81.3333333333333
Norman: 61.9381625441696
Mahnomen: 63.1228813559322
Cass: 64.3214285714286
Hubbard: 52.8985507246377
Clay: 60.2283205268935
Becker: 59.3591549295774
Aitkin: 78.2857142857143
Wadena: 63.175
Crow Wing: 76.3333333333333
Carlton: 70.66666666666

Olmsted ET map for 2018 has been processed and saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\May 2021 ET\May2018ET\Olmsted_ET_2018.tif. Average value: 86.1984496124032
Watonwan ET map for 2018 has been processed and saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\May 2021 ET\May2018ET\Watonwan_ET_2018.tif. Average value: 72.5232067510548
Rock ET map for 2018 has been processed and saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\May 2021 ET\May2018ET\Rock_ET_2018.tif. Average value: 55.707182320442
Nobles ET map for 2018 has been processed and saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\May 2021 ET\May2018ET\Nobles_ET_2018.tif. Average value: 57.2348754448399
Jackson ET map for 2018 has been processed and saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\May 2021 ET\May2018ET\Jackson_ET_2018.tif. Average value: 66.6241860465116
Martin ET map for 2018 has been processed and saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projec